<a href="https://colab.research.google.com/github/zeeshansalim1234/CalgaryHacks-Hackathon/blob/main/Pachira_nan_reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing & Importing the required libraries


In [ ]:

!pip install sentence-transformers
!pip install tweepy
!pip install bert-extractive-summarizer
!pip install nltk
!pip install google-cloud-vision

In [29]:
import pandas as pd
import json
import os
from sentence_transformers import SentenceTransformer, util
from google.colab import drive
import tweepy
import re
import os,io
from google.cloud import vision
from google.cloud.vision_v1 import types
import pandas as pd
import nltk

ACCESS_TOKEN="1401579178072231936-98mY1wOw1UR3GHOjdo4ePnfdfUHt6n"
ACESS_TOKEN_SECRET="QiIslicgx8GNGrGDPrx8xNZc2WoCyrUsZumgXSyGzGLOx"
CONSUMER_KEY="uaiStuv7EYdYxWSomPKHvSSF5"
CONSUMER_SECRET="JOQryy37w9HPMSrSw8msyyb048iqeHmK4xCRyWP1oBhLKwLYlb"

auth=tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN,ACESS_TOKEN_SECRET)
api=tweepy.API(auth)

#os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'seismic-diorama-316110-5569927e0d86.json'
#client = vision.ImageAnnotatorClient()

# Function for pre-processing data (A list of Strings should be passed as the argument)

In [30]:
def pre_process(tweets):

    for i in range(0, len(tweets)):

        if (tweets[i] is not None):

            if(tweets[i]!=tweets[i]):
                tweets[i]=""

            tweets[i] = tweets[i].lower()  # To lower case
            tweets[i] = tweets[i].replace('@','')  # remove @
            tweets[i] = tweets[i].replace('#','')  # remove #
            tweets[i] = remove_urls(tweets[i])  # remove URL
            tweets[i] = remove_emojis(tweets[i])  # remove emojis
            tweets[i] = "".join(j for j in tweets[i] if j not in (
            "?", ".", ";", ":", "!", "-", ",", "[", "]", "(", ")", "’", "‘", '"', "$", "'", "“", "”", "•", "=", "+",
            "%", "/", "&", "|", "~"))  # remove punctuations
            tweets[i] = removeNonEnglishWordsFunct(tweets[i])

    return tweets

def remove_urls (str):    # Removes URL from strings

    str = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str, flags=re.MULTILINE)
    return(str)


def remove_emojis(data):  # Removes Emojis from strings

    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoji, '', data)

def removeNonEnglishWordsFunct(x):  # Remove non-rnglish characters from strings

    new_string=re.sub('[^a-zA-Z0-9]',' ',x)

    cleaned_string=re.sub('\s+',' ',new_string)
    return cleaned_string

# Extracting charitable organisation data from Pachira dataset and generating embedding using Specter ML model

In [31]:
import numpy as np
df = pd.read_csv('/content/charities_bc.csv') #reading from csv file and creating a pandas dataset

df = df.dropna() #dropping blank rows

charities=df.CharityName.values+" "+df.OngoingPrograms.values # charities list hold the organisation names and description
cities=df.City.values #cities list holds city names

print(len(charities), "Charities")

#We then load the allenai-specter model with SentenceTransformers
model = SentenceTransformer('allenai-specter')   # loading specter model

#To encode the descriptions to a single string
charity_texts = [charity for charity in charities]
cities_texts=[city for city in cities]

#Compute embeddings for all descriptions
corpus_embeddings = model.encode(charity_texts, convert_to_tensor=True)  # generating embeddings using charities name and description


2086 Charities


# Function to find all charities in the same location which is passed as the argument (pass a string as location)

In [33]:

# Returns all charities at the provided location

def search_papers_location(location):

   results=[]
   for i in range(0,len(cities_texts)):
     if(cities_texts[i]==location):
       results.append(i)
   return results

  

# Randomly generating 100 priority charities (Replace with actual priority charities when provided by Pachira)

In [34]:
# Generates 100 random charities as the priority

import random
from termcolor import colored

#Generate 5 random numbers between 10 and 30
priority = random.sample(range(0, 1520), 100)
print(priority)


[72, 661, 527, 442, 253, 11, 301, 1091, 776, 1076, 634, 696, 1211, 749, 733, 1477, 693, 1145, 1426, 216, 880, 95, 662, 556, 1317, 64, 1413, 1090, 476, 470, 1151, 969, 1143, 312, 1294, 1349, 1231, 58, 365, 43, 677, 896, 479, 653, 808, 1024, 382, 397, 897, 823, 1221, 1333, 353, 310, 756, 1017, 658, 417, 739, 1303, 1141, 533, 1239, 795, 906, 143, 167, 692, 491, 245, 1069, 744, 295, 1133, 1433, 690, 440, 481, 406, 1099, 856, 667, 1192, 968, 1453, 264, 1467, 186, 196, 1012, 622, 1071, 977, 355, 1118, 738, 457, 618, 114, 583]


# Stores the charities recommendations for the passed string argument in a list (Only for priority charities)

In [35]:
# For generating recommendations for priority organizations

def generate_priority(title):

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10) # Generates a 2D matrix that contains the id and similarity score of the top 10 matching charities
   search_hits = search_hits[0]
   
   count = 0
   top_header = "\n\nPriority Charities\n\n"
   print(colored(top_header,'yellow'))

   flag=0

   for hit in search_hits:

          for i in range(0,len(priority)):
            if(hit['corpus_id']==priority[i]):   # checks if the charity in top 10 is a part of the priority charities
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>0.70):
            related_charities = charities[hit['corpus_id']]
            count += 1
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values

            print("\n"+str(count)+") "+colored(str(k[0][0]), 'red'))
            print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            flag=0
            count+=1


# Stores the charities recommendations for the passed string argument in a list (for all charities)

In [52]:

# For Generating recommendation for all organizations

def search_papers(title,city_indices,status,num_results):

   #status signifies city 0 is city based
   #num_results signifies the number of results we wish to generate

   query_embedding = model.encode(title+'[SEP]', convert_to_tensor=True) # Converts to tensor
   search_hits = util.semantic_search(query_embedding, corpus_embeddings,top_k=10)
   search_hits = search_hits[0]
   
   count = 0
   flag=0
   tracker=0 #basically counter for the non location segment

   check_numreco_tracker=0  #to track the number of location based reco

   for hit in search_hits:
        
        if(status==0) :

          for i in range(0,len(city_indices)):
            if(hit['corpus_id']==city_indices[i]):
              flag=1
              break

          if(flag==1 and search_hits[count]['score']>=0.80):

            related_charities = charities[hit['corpus_id']]
            subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms']== related_charities]
            k=subsetDataFrame.values
            

            print("\n"+str(check_numreco_tracker+1)+") "+colored(str(k[0][0]), 'red'))
            all_names.append(str(k[0][0]))
            print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
            all_scores.append(search_hits[count]['score'])
            print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
            all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
            check_numreco_tracker+=1 #to add the number of reco with location
            flag=0

        else:

          if(tracker<num_results): #For normal non-location based (passed blank array in the function call at end) 

            if(len(city_indices)==0 and search_hits[count]['score']>=0.80):

              related_charities = charities[hit['corpus_id']]
              subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms'] == related_charities]
              k=subsetDataFrame.values       
              print("\n"+str(tracker+1)+") "+colored(str(k[0][0]), 'red'))
              all_names.append(str(k[0][0]))
              print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
              all_scores.append(search_hits[count]['score'])
              print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
              all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
              flag=0
              tracker+=1

            elif(len(city_indices)!=0 and cities_texts[hit['corpus_id']]!=cities_texts[city_indices[0]]): #for recursice location based (non-location recos)

              related_charities = charities[hit['corpus_id']]
              subsetDataFrame = df[df['CharityName']+" "+df['OngoingPrograms'] == related_charities]
              k=subsetDataFrame.values       
              print("\n"+str(6-num_results+tracker)+") "+colored(str(k[0][0]), 'red'))
              all_names.append(str(k[0][0]))
              print("similiarity score of " + str(format(search_hits[count]['score'],".2f")))
              all_scores.append(search_hits[count]['score'])
              print("Description of charity : " +related_charities+" "+cities_texts[hit['corpus_id']])
              all_descriptions.append(related_charities+" "+cities_texts[hit['corpus_id']])
              flag=0
              tracker+=1

        count+=1 #keep track of elements in search_hits

   if(status==0): 

     search_papers(title,city_indices,1,5-check_numreco_tracker)  # now sending non-location based reco if didnt reach top 5 already


#search_papers("alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary",[],1,10)

#want to make a big impact donations made through birdies4kids for the alberta diabetes foundation are matched up to 50  birdies for kids runs until august 15th 2021  to donate and read more 

#alberta wins reddeer sisters donate land to nature conservancy of canada ncc preserve a haven for species wildlife ruth dorothy bower donate 193 hectares of land on the west bank of the reddeer river bowerwildlifesanctuary

#in the news the kidney foundation encourages other canadian organizations and companies to to adopt policies to support living organdonors read more about kidneycanada s wage replacement policy for living organ and tissue donation

#participate in the scotiabank calgary marathon charity challenge to help children amp families at childrens cottage society theres an event for everyone of all abilities register or donate at  thanks for your consideration children families 

#can you help us many children need their very own books strong reading role models amp safe fun reading spaces your donation helps more children experience the magic of reading amp a lifetime of opportunity 

# Using Google PyTrends API to get trending topics

In [37]:
pip install pytrends


In [38]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US') #specify language

"""
data=pytrends.trending_searches('canada')
print(data[0].loc[0])
"""

pytrends.build_payload(kw_list=['protest'],geo='CA',timeframe='today 1-m')  # generate the query
related_queries = pytrends.related_queries()
result=list(related_queries.values())
query=list(result[0].values())[1]
#query="Women's soccer final"
print(query)

#search_papers(query,[],1,5)

#list(result[0].values())[1].loc[0].at['query']

None


In [39]:
#df = pytrends.trending_searches(pn='canada')
#df.head()

# Menu to generate :
#  1. Trending Charities Recommendation
#  2. Priority Charities Recommendation
#  3. General Chairities Recommendation

#  for each individual sentence provided from a conversation.

In [54]:
print("What basis do you want to generate a recommendation on ? ")
print("\n1. Enter a sentence  ")
print("2. Trending tweet on a particular hashtag ")
choice=input("Enter your choice : ")

if(choice=="1"):

  title=input("\nEnter a sentence : ")
  enable_loc=str(input("Do you wish to enable location ? "))
  if(enable_loc=="yes"):
    location=str(input("Enter location : "))

  pytrends = TrendReq(hl='en-US') #specify language
  pytrends.build_payload(kw_list=['nature'],geo='CA',timeframe="now 7-d")  # generate the query
  related_queries = pytrends.related_queries()
  result=list(related_queries.values())
  query=list(result[0].values())[1].loc[2].at['query']
  print("\n"+query)
  
  top_header = "\n\nPopular in your Region\n\n"
  print(colored(top_header,'magenta'))
  #search_papers(query,[],1,5)
  

  if(enable_loc=="yes"):

    status=0
    loc_indices=search_papers_location(location)
    generate_priority(title)
    top_header = "\n\nTop related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,loc_indices,status,10)

  else:

    
    temp=[]
    status=1
    generate_priority(title)
    top_header = "\n\nTop related charities\n\n"
    print(colored(top_header,'blue'))
    search_papers(title,temp,status,10)

elif(choice=="2"):

  tag = input("\nEnter a hashtag : ")
  tweets=tweepy.Cursor(api.search,q=tag,result_type='popular',tweet_mode="extended").items(1)
  temp=[]
  for tweet in tweets :

      temp.append(tweet.full_text)

  temp=pre_process(temp)
  tweet=temp[0]
  print("\nMost famous tweet : \n"+colored(tweet,'orange'))
  search_papers(tweet)


What basis do you want to generate a recommendation on ? 

1. Enter a sentence  
2. Trending tweet on a particular hashtag 
Enter your choice : 1

Enter a sentence : in the news the kidney foundation encourages other canadian organizations and companies to to adopt policies to support living organdonors read more about kidneycanada s wage replacement policy for living organ and tissue donation
Do you wish to enable location ? no

where was nature of love filmed


Popular in your Region




Priority Charities




Top related charities



1) BRUCE DENNISTON BONE MARROW SOCIETY
similiarity score of 0.85
Description of charity : BRUCE DENNISTON BONE MARROW SOCIETY SUPPORT OF CANADIAN BLOOD SERVICES' STEM CELL REGISTRY, BY EDUCATING THE PUBLIC ABOUT THE NEED FOR REGISTERED DONORS. OUR VOLUNTEER STEM CELL AND BONE MARROW COURIERS TRAVEL THE WORLD TO TRANSPORT STEM CELLS TO HOSPITALS IN CANADA WHERE PATIENTS ARE AWAITING A LIFE-SAVING STEM CELL OR BONE MORROW TRANSPLANT. SHIRTS DESIGNED TO AC

# Extracting artificial conversation history created for a user (**To be replaced with actual user history from Pachira**)

In [41]:

df_history = pd.read_csv('/content/artificial_conversation.csv',sep=";", encoding='mac_roman')

#charities =df.CharityName.values+" "+df.OngoingPrograms.values

print(str(df_history.columns[0]))

conversations=df_history.Sentence.values

print(conversations)



Sentence
['Forest-dependent communities in Laos make almost all of their livelihoods from their forestsí natural resources. Find out how they started seedling nurseries and planting initiatives to restore their essential forests in partnership with WWF-Laos.'
 'NSW recorded 356 new locally acquired cases of COVID-19 in the 24 hours to 8pm last night.'
 "Thank you, everyone! We raised $232 during today's charity punishment stream. Our donation was enough to send 4 girls to school for a year, thanks to the International Rescue Committee! We also were able to gift a few steam gift cards to chat &have a fun day!"
 'The military junta is the root cause of humanitarian crisis in #Myanmar. The junta has cut all supply routes to #Mindat & IDPs are now at risk of food shortage. In July, the junta has prevented #UNHCR \n@Refugees\n from delivering aid to IDPs in #Chin State.'
 'Vegetable producer says it is losing thousands of pounds a week and throwing away food due to labour shortage'
 'Figure

# Storing the recommendations from each conversation in the history dataset to be filtered later

In [43]:
all_names=[]
all_descriptions=[]
all_scores=[]

for i in range(0,len(conversations)):
  print(colored(conversations[i],'magenta'))
  search_papers(conversations[i],[],1,10)
  print("\n\n\n\n\n")



print(all_names)
print(all_scores)

Forest-dependent communities in Laos make almost all of their livelihoods from their forestsí natural resources. Find out how they started seedling nurseries and planting initiatives to restore their essential forests in partnership with WWF-Laos.

1) THE FOREST BIRD SOCIETY
similiarity score of 0.90
Description of charity : THE FOREST BIRD SOCIETY THE ORGANIZATION IS WINDING DOWN DEVELOPMENT OF PROJECTS AND WILL CONCENTRATE IN TRYING TO IMPLEMENT SUCH PROJECTS, IT IS ACTIVELY SEEKING NEW FUNDING FROM NOW ONWARDS. RICHMOND

2) E.I.S.-THE ECOFORESTRY INSTITUTE SOCIETY
similiarity score of 0.90
Description of charity : E.I.S.-THE ECOFORESTRY INSTITUTE SOCIETY * Resources and expertise were provided to develop sustainable land use and management plans related to ecologically oriented forestry.*  As holder of the WIldwood Trust,  Wildwood is maintained as per an Ecoforestry Management Plan* 5000 volunteer hours were logged (as we have no staff) to provide education tours to school and univ

In [55]:
charity_counter=0
temp_scores=[]
indices=[]
top_names=[]
top_descriptions=[]

for i in range(0,len(all_scores)):
  temp_scores.append(all_scores[i])

all_scores = sorted(all_scores, reverse=True) 



# Showing only top 10 recommendations chosen from all the recommendations generated for different conversations on the basis of the highest similarity scores of the recommendations

## **- This approach was taken to mantain diversity in recommendations while having high quality recommendations (highest degree of textual match)**

## **- This approach can be changed if a sutaible alternate method is devised**

In [56]:


for i in range(0,10):
  for j in range(0,len(temp_scores)):
    if(all_scores[i]==temp_scores[j]):
      indices.append(j)

print(indices)
for i in range(0, 10):
  top_names.append(all_names[indices[i]])
  top_descriptions.append(all_descriptions[indices[i]])




[0, 53, 1, 54, 2, 55, 56, 57, 58, 59]


In [46]:
counter=0

choice=input("Want to set a location? ")
print("\n")

if(choice=="yes"):
  user_location=input("Enter City: ")
  print("\n")
  for i in range(0,10):
    if(user_location in top_descriptions[i]):
      print(str(counter+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
      print(top_descriptions[i])
      print("\n")
      counter+=1
    
  for i in range(0,10):
    if(user_location not in top_descriptions[i]):
      print(str(counter+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
      print(top_descriptions[i])
      print("\n")
      counter+=1
else:
   for i in range(0,10):
  
    print(str(i+1)+") "+colored(top_names[i],'red')+"("+str(temp_scores[i])+")")
    print(top_descriptions[i])
    print("\n")
     

Want to set a location? yes


Enter City: Vancouver


1) SAVE YOUR SKIN FOUNDATION(0.904597818851471)
SAVE YOUR SKIN FOUNDATION EDUCATION + AWARENESS OF SKIN CANCER AT COMMUNITY EVENTS, SOCIAL MEDIA, WEBINARS, MULTI STAKEHOLDER EVENTS TO EDUCATE ON INNOVATIVE MEDICINES  - PATIENT SUPPORT MEETINGS NORTH VANCOUVER


2) THE FOREST BIRD SOCIETY(0.9025729298591614)
THE FOREST BIRD SOCIETY THE ORGANIZATION IS WINDING DOWN DEVELOPMENT OF PROJECTS AND WILL CONCENTRATE IN TRYING TO IMPLEMENT SUCH PROJECTS, IT IS ACTIVELY SEEKING NEW FUNDING FROM NOW ONWARDS. RICHMOND


3) E.I.S.-THE ECOFORESTRY INSTITUTE SOCIETY(0.8728896975517273)
E.I.S.-THE ECOFORESTRY INSTITUTE SOCIETY * Resources and expertise were provided to develop sustainable land use and management plans related to ecologically oriented forestry.*  As holder of the WIldwood Trust,  Wildwood is maintained as per an Ecoforestry Management Plan* 5000 volunteer hours were logged (as we have no staff) to provide education tours to school an